## 定义采用向量化的梯度下降方法

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 定义计算损失函数值得方法
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2)
    except:
        return float('inf')

In [3]:
# 定义计算导数值得方法，使用向量化的方式取代之前的 for 循环
def dJ(theta, X_b, y):
#     result = np.empty(len(theta))
#     result[0] = np.sum(X_b.dot(theta) - y)
#     for i in range(1, len(theta)):
#         result[i] = np.sum((X_b.dot(theta) - y).dot(X_b[:, i]))
#     return result * 2 / len(X_b)
    return X_b.T.dot(X_b.dot(theta) -y) * 2 / len(X_b)

In [4]:
# 梯度下降的过程包装成方法
def gradient_descent(X_b, y, init_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = init_theta
    i_iters = 1
    while i_iters <= n_iters:
        # 首先求当前点的梯度
        gradient = dJ(theta, X_b, y)
        # 记下移动前的位置
        last_theta = theta
        # 往函数值小的方向移动
        theta = theta - eta * gradient
        # 退出机制
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        i_iters += 1
    print('total steps:', i_iters)
    return theta

## 把上面的方法包装到自己的 LinearRegression.py 中，和正规方程的线性回归做比较
代码：playML06/LinaerRegression.py

## 使用波士顿房价的数据 来测试

In [5]:
from sklearn import datasets

In [6]:
boston = datasets.load_boston()
X = boston.data
y = boston.target
X = X[y < 50.0]
y = y[y < 50.0]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(392, 13)
(392,)
(98, 13)
(98,)


In [8]:
# 使用自己包装的线性回归算法类
from playML06.LinearRegression import LinearRegression

In [9]:
# 先使用正规方程来解线性回归
reg1 = LinearRegression()
reg1.fit_normal(X_train, y_train)
reg1.score(X_test, y_test)

0.8129794056212907

In [10]:
# 在使用梯度下降法前，必须对数据归一化，因为各特征的数据量级间存在很大的差异，梯度下降过程中可能会导致溢出或者循环次数过大
# 而上面的正规方程做拟合就不需要数据归一化，因为它是一个很明确标准的公式，它自己可以全部消化。
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
standardScaler.fit(X_train)
X_train_standard = standardScaler.transform(X_train)
X_test_standard = standardScaler.transform(X_test)

In [11]:
# 使用自己包装的线性回归算法类
reg2 = LinearRegression()
reg2.fit_gd(X_train_standard, y_train)

total steps: 6824


LinearRegression()

In [12]:
reg2.score(X_test_standard, y_test)

0.8129798083983443

In [13]:
# 这个分数和之前用线性回归正规方程得到的值差不多。

## 梯度下降的优势

In [14]:
# 相对于线性回归正规方程的解法，梯度下降法对于特征数量巨大时能体现出巨大的性能优势，主要得益于矩阵的向量化运算速度优势

In [15]:
m = 1000
n = 5000

big_X = np.random.normal(size=(m, n))
# 真实参数用一个均匀分布产生
true_theta = np.random.uniform(0, 100, size=n+1)
big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0.0, 10.0, size=m)

In [16]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

CPU times: user 14.5 s, sys: 361 ms, total: 14.8 s
Wall time: 1.88 s


LinearRegression()

In [17]:
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y)

total steps: 523
CPU times: user 31.8 s, sys: 288 ms, total: 32.1 s
Wall time: 4.03 s


LinearRegression()

### 但是上面的实际测试结果和bobo老师运行出来的不一样，我这里梯度下降比正规方程慢很多？？ 可能是我的机器CPU核树比较多，导致这个差异